In [1]:
import json
import os

from easydict import EasyDict as edict
import redis

In [2]:
ROOT_PATH   = '/'.join(os.getcwd().split(os.path.sep)[:-4])
CONFIG_PATH = f'{ROOT_PATH}/utils/configs/config.json'
PORTS_PATH  = f'{ROOT_PATH}/utils/configs/ports.json'

CONFIG      = edict(json.loads(open(CONFIG_PATH, 'r').read()))
PORTS       = edict(json.loads(open(PORTS_PATH, 'r').read()))

In [3]:
host        = CONFIG.global_host
port        = PORTS.redis_port

conn        = redis.Redis(host = host, port = port, decode_responses = True)
conn.ping()

True

In [4]:
## DB 안에 저장되어 있는 key 값들을 조회하기 위해서 keys를 이용

#! DB 안에 저장되어 있는 key 값들이 많은 경우 오랜 시간이 걸리고, 다른 명령을 처리하지 못하기 때문에
#! redis server를 운영 중에는 전체 키를 조회하는 keys *를 사용해서는 안된다.

#! 대신 scan 명령어로 일정 수의 키를 반복 수행 (한 번에 약 10개씩 조회)
print(conn.scan())
## 값을 하나씩 저장하기 위해서는 set을 이용
conn.set('Dove', 'Kim')
print(conn.scan())

## 값을 하나씩 가져오기 위해서는 get을 이용
conn.get('Dove')

(0, ['animals', 'birds', 'Dove'])
(0, ['animals', 'birds', 'Dove'])


'Kim'

In [5]:
## 한 번에 여러개의 키, 값을 넣기 위해서는 데이터를 
## 딕셔너리에 넣고mset을 이용
data = {
            'Pigeon' : 'Lee',
            'Lion'   : 'Park'
        } 

conn.mset(data)
print(conn.scan())

## 한 번에 여러개의 키를 이용하여 값을 얻기 위해서는
## mget을 이용
conn.mget('Dove', 'Lion')

(0, ['animals', 'birds', 'Lion', 'Dove', 'Pigeon'])


['Kim', 'Park']

In [6]:
## 데이터를 삭제하기 위해서는 delete을 이용
#* 한 번에 여러개도 삭제 가능
conn.delete('Pigeon', 'Lion')
conn.scan()

(0, ['animals', 'birds', 'Dove'])

In [7]:
## list를 이용하여 하나의 키에 여러개의 값을 넣기 위해선 lpush를 이용한다.
birds = ['sparrow', 'raven', 'crow', 'pigeon']
conn.lpush('birds', *birds)
print(conn.scan())

## list 형을 값으로 가지고 있는 데이터를 조회하기 위해서는 lrange를 이용한다.
#! lrange('조회하려는 키', 시작 인덱스, 끝 인덱스)
print(conn.lrange('birds', 0, -1))
print(conn.lrange('birds', 1, 2))
print()
## set 데이터를 입력하기 위해서는 sadd를 이용한다.
#! 이 때는 중복된 데이터는 하나로 합쳐진다.
animals = ['tiger', 'hippo', 'lion', 'lion']
conn.sadd('animals', *animals)
print(conn.scan())

## set 데이터를 값으로 가지고 있는 데이터를 조회하기 위해서는 smember를 이용한다.
print(conn.smembers('animals'))

(0, ['animals', 'birds', 'Dove'])
['pigeon', 'crow', 'raven', 'sparrow', 'pigeon', 'crow', 'raven', 'sparrow']
['crow', 'raven']

(0, ['animals', 'birds', 'Dove'])
{'lion', 'hippo', 'tiger'}
